In [ ]:
from sklearn.metrics import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import random

## Loading Data

In [ ]:
import pandas as pd

In [ ]:
DATA_FOLDER = "Data_Processed/Shared_Task_eng/"

In [ ]:
df_train = pd.read_csv(DATA_FOLDER+"train_1.csv")
df_val = pd.read_csv(DATA_FOLDER+"val_1.csv")
df_test = pd.read_csv(DATA_FOLDER+"test_1.csv")

In [ ]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)
df_test.dropna(inplace=True)

In [ ]:
len(df_train)

## Word Embeddings
- https://towardsdatascience.com/nlp-embedding-techniques-51b7e6ec9f92

In [ ]:
X_train = df_train['Text'].values
Y_train = df_train['Label'].values
X_val = df_val['Text'].values
Y_val = df_val['Label'].values
X_test = df_test['Text'].values
Y_test = df_test['Label'].values

In [ ]:
X_train.shape,X_val.shape,X_test.shape

### Bag of Words

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

In [ ]:
vector = vectorizer.transform(X_train)

In [ ]:
vector.shape

In [ ]:
def get_bow_embedding(train,val,test):
    vectorizer = CountVectorizer()
    vectorizer.fit(train)
    
    x_train = vectorizer.transform(train)
    x_val = vectorizer.transform(val)
    x_test = vectorizer.transform(test)
    
    return x_train,x_val,x_test

In [ ]:
x_train,x_val,x_test = get_bow_embedding(X_train,X_val,X_test)

In [ ]:
x_train.shape,x_val.shape,x_test.shape

### Tf-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer_tfidf = TfidfVectorizer()

In [ ]:
vectorizer_tfidf.fit(X_train)

In [ ]:
vector_tfidf = vectorizer_tfidf.transform(X_train)

In [ ]:
vector_tfidf.shape

In [ ]:
def get_tfidf_embedding(train,val,test):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(train)
    
    x_train = vectorizer.transform(train)
    x_val = vectorizer.transform(val)
    x_test = vectorizer.transform(test)
    
    return x_train,x_val,x_test

In [ ]:
x_train,x_val,x_test = get_tfidf_embedding(X_train,X_val,X_test)

In [ ]:
x_train.shape,x_val.shape,x_test.shape

### Google's Idea
- Ref: https://developers.google.com/machine-learning/guides/text-classification/step-3
- Convert Words into uni-grams and bi-grams and calculate all features using tf-idf and then select top 20K features using f_classif or chi_2
- This article says that normalisation does not help text datasets much so it is not used here

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

In [ ]:
kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'float32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }

In [ ]:
def ngram_vectorise(kwargs,train_text,train_labels,val_text,test_text):
    vectorizer = TfidfVectorizer(**kwargs)
    
    x_train = vectorizer.fit_transform(train_text)
    
    x_val = vectorizer.transform(val_text)
    x_test = vectorizer.transform(test_text)
    
    print("Features Calculated: ",x_train.shape[1])
    
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    x_test = selector.transform(x_test).astype('float32')
    
    return x_train,x_val,x_test

In [ ]:
x_train,x_val,x_test = ngram_vectorise(kwargs,X_train,Y_train,X_val,X_test)

In [ ]:
x_train.shape,x_val.shape,x_test.shape

### Word2Vec
- Ref: https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api

In [ ]:
wv = api.load('word2vec-google-news-300')

In [ ]:
wv['gold'].shape

In [ ]:
def encode_w2v(wv,data):
    new_data=[]
    for sentence in data:
        sample=[]
        for word in sentence:
            if word in wv:
                sample.append(wv[word])
            else:
                sample.append(np.zeros(shape=(300)))
        new_data.append(np.mean(np.array(sample),axis=0))
    return np.array(new_data)

In [ ]:
x_train_word2vec = get_word2vec_embeddings(wv,X_train)

In [ ]:
x_train_word2vec.shape

In [ ]:
def get_w2v_embedding(wv,train,val,test):
    x_train = encode_w2v(wv,train)
    x_val = encode_w2v(wv,val)
    x_test = encode_w2v(wv,test)
    
    return x_test,x_val,x_test

In [ ]:
x_train,x_val,x_test = get_w2v_embedding(wv,X_train,X_val,X_test)

In [ ]:
x_train.shape,x_val.shape,x_test.shape

### Doc2Vec
- https://radimrehurek.com/gensim/models/doc2vec.html

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X_train)]

In [ ]:
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4)

In [ ]:
def encode_d2v(model,data):
    embed = [list(model.infer_vector(ele.split('.'))) for ele in data]
    return np.array(embed)

In [ ]:
def get_d2v_embedding(model,train,val,test):
    x_train = encode_d2v(model,train)
    x_val = encode_d2v(model,val)
    x_test = encode_d2v(model,test)
    
    return x_train,x_val,x_test

In [ ]:
x_train,x_val,x_test = get_d2v_embedding(model,X_train,X_val,X_test)

In [ ]:
x_train.shape,x_val.shape,x_test.shape

## Laser

In [ ]:
!pip install laserembeddings

In [ ]:
!python -m laserembeddings download-models

In [ ]:
from laserembeddings import Laser

In [ ]:
laser= Laser()

In [ ]:
def encode_laser(train,test,laser,l):
    x_train = laser.embed_sentences(train)
    x_test = laser.embed_sentences(test)
    
    return x_train,x_test

In [ ]:
X_train = df_train['Text'].values
Y_train = df_train['Label'].values
X_val = df_val['Text'].values
Y_val = df_val['Label'].values
X_test = df_test['Text'].values
Y_test = df_test['Label'].values

In [ ]:
x_train,x_val = encode_laser(X_train,X_test,laser)

## Models
- Logistic Regression
- SVM
- KNN
- Gaussian NB
- Decision Tree
- Random Forest
- XGBoost
- AdaBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier

In [ ]:
def evalMetric(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    mf1Score = f1_score(y_true, y_pred, average='macro')
    f1Score  = f1_score(y_true, y_pred, labels = np.unique(y_pred))
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    area_under_c = auc(fpr, tpr)
    recallScore = recall_score(y_true, y_pred, labels = np.unique(y_pred))
    precisionScore = precision_score(y_true, y_pred, labels = np.unique(y_pred))
    return dict({"accuracy": accuracy, 'mF1Score': mf1Score, 
                    'f1Score': f1Score, 'precision': precisionScore, 
                    'recall': recallScore})

In [ ]:
weights = {0:1,1:8}

### SVM

In [ ]:
SVM = SVC(kernel="rbf",class_weight=weights,probability=True)
SVM.fit(x_train,Y_train)
y_pred = SVM.predict(x_val)

In [ ]:
evalMetric(Y_val,y_pred)

In [ ]:
DT = DecisionTreeClassifier(class_weight=weights, 
            criterion='gini', max_depth=100, max_features=1.0, 
            max_leaf_nodes=10, min_impurity_split=1e-07, 
            min_samples_leaf=1, min_samples_split=2, 
            min_weight_fraction_leaf=0.10, presort=False, 
            random_state=42, splitter='best')

In [ ]:
SVM = SVC(kernel="rbf",class_weight=weights,probability=True)

In [ ]:
RF = RandomForestClassifier(n_estimators=1000,random_state=0,
                    n_jobs=1000,max_depth=100,bootstrap=True,
                      class_weight=weights)

In [ ]:
AB =AdaBoostClassifier()

In [ ]:
KNN = KNeighborsClassifier(leaf_size=1,p=2,n_neighbors=20)

In [ ]:
GNB = GaussianNB()

In [ ]:
LR = LogisticRegression(C=1.0, class_weight=weights, dual=False, 
        fit_intercept=True, intercept_scaling=1, max_iter=100, 
        multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, 
        solver='liblinear', tol=0.0001,verbose=0, warm_start=False)

In [ ]:
XGBC = XGBClassifier(learning_rate =0.1, n_estimators=100000, 
            max_depth=6, min_child_weight=6, gamma=0, subsample=0.6, 
            colsample_bytree=0.8, reg_alpha=0.005, 
            objective= 'binary:logistic', nthread=2, 
            scale_pos_weight=1, seed=42, class_weight=weights)

In [ ]:
classifiers=[SVM,XGBC,RF,DT,AB,KNN,GNB,LR]

In [ ]:
names = ['SVC','XGBoost','Random Forest','Decision Tree','AdaBoost',
        'KNN','Gausian NB','Logistic Regression']

In [ ]:
res={}

In [ ]:
for i,cf in enumerate(classifiers):
    cf.fit(x_train,Y_train)
    y_pred = cf.predict(x_val)
    metric = evalMetric(Y_val,y_pred)
    res[names[i]]=metric

In [ ]:
res